## Perguntas

1) Correlação das variaveis
2) Qual estilo de jogo é mais provavel de se ter melhores vendas?
3) Qual plataforma é provavel de se ter melhores vendas?
4)  Aplicar um modelo de regressão ou classificação utilizando uma técnica de hiperparametrização automática;</br>
5)  Aplicar uma medida de avaliação do modelo.
6)  Agrupamento: gerar a visualização Scatter
7)  Classificação: gerar a matriz de confusão
8)  Regressão: gerar a visualização da linha de saída

In [16]:
#IMPORT E INSTALL AS BIBLIOTECAS IMPORTANTES
!pip install googletrans==4.0.0-rc1

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from googletrans import Translator

In [43]:
caminho = 'Data/vgchartz-2024.csv'
jogos = pd.read_csv(caminho)

In [44]:
jogos.head(10)

,img,title,console,genre,publisher,developer,critic_score,total_sales,na_sales,jp_sales,pal_sales,other_sales,release_date,last_update
0,/games/boxart/full_6510540AmericaFrontccc.jpg,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,9.4,20.32,6.37,0.99,9.85,3.12,2013-09-17,NaN
1,/games/boxart/full_5563178AmericaFrontccc.jpg,Grand Theft Auto V,PS4,Action,Rockstar Games,Rockstar North,9.7,19.39,6.06,0.60,9.71,3.02,2014-11-18,2018-01-03
2,/games/boxart/827563ccc.jpg,Grand Theft Auto: Vice City,PS2,Action,Rockstar Games,Rockstar North,9.6,16.15,8.41,0.47,5.49,1.78,2002-10-28,NaN
3,/games/boxart/full_9218923AmericaFrontccc.jpg,Grand Theft Auto V,X360,Action,Rockstar Games,Rockstar North,NaN,15.86,9.06,0.06,5.33,1.42,2013-09-17,NaN
4,/games/boxart/full_4990510AmericaFrontccc.jpg,Call of Duty: Black Ops 3,PS4,Shooter,Activision,Treyarch,8.1,15.09,6.18,0.41,6.05,2.44,2015-11-06,2018-01-14
5,/games/boxart/full_call-of-duty-modern-warfare...,Call of Duty: Modern Warfare 3,X360,Shooter,Activision,Infinity Ward,8.7,14.82,9.07,0.13,4.29,1.33,2011-11-08,NaN
6,/games/boxart/full_call-of-duty-black-ops_5Ame...,Call of Duty: Black Ops,X360,Shooter,Activision,Treyarch,8.8,14.74,9.76,0.11,3.73,1.14,2010-11-09,NaN
7,/games/boxart/full_4653215AmericaFrontccc.jpg,Red Dead Redemption 2,PS4,Action-Adventure,Rockstar Games,Rockstar Games,9.8,13.94,5.26,0.21,6.21,2.26,2018-10-26,2018-11-02
8,/games/boxart/full_1977964AmericaFrontccc.jpg,Call of Duty: Black Ops II,X360,Shooter,Activision,Treyarch,8.4,13.86,8.27,0.07,4.32,1.20,2012-11-13,2018-04-07
9,/games/boxart/full_4649679AmericaFrontccc.png,Call of Duty: Black Ops II,PS3,Shooter,Activision,Treyarch,8.0,13.80,4.99,0.65,5.88,2.28,2012-11-13,2018-04-07


In [45]:
jogos = jogos.drop(columns=['img'])

In [42]:
#TRADUZIR OS NOMES EM INGLES PARA PT-BR

def traduzir_colunas(df, language='pt'):
    translator = Translator()
    traducoes = {}
    # Traduzir todas as colunas
    for column in df.columns:
        try:
            translated = translator.translate(column, src='en', dest=language)
            capslock = translated.text.capitalize()  # Maiuscula na primeira letra de cada palavra
            traducoes[column] = capslock
        except Exception as e:
            print(f"Erro ao traduzir a coluna '{column}': {e}")
            traducoes[column] = column  # Manter o nome original em caso de erro
    df.columns = [traducoes.get(col, col) for col in df.columns]

In [47]:
traduzir_colunas(jogos)
jogos.head()

Erro ao traduzir a coluna 'publisher': 'NoneType' object is not iterable
Erro ao traduzir a coluna 'developer': 'NoneType' object is not iterable


,Título,Console,Gênero,publisher,developer,Critic_score,Vendas totais,Na_sales,Jp_sales,Pal_sales,Outros_sales,Dados de lançamento,Última atualização
0,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,9.4,20.32,6.37,0.99,9.85,3.12,2013-09-17,NaN
1,Grand Theft Auto V,PS4,Action,Rockstar Games,Rockstar North,9.7,19.39,6.06,0.60,9.71,3.02,2014-11-18,2018-01-03
2,Grand Theft Auto: Vice City,PS2,Action,Rockstar Games,Rockstar North,9.6,16.15,8.41,0.47,5.49,1.78,2002-10-28,NaN
3,Grand Theft Auto V,X360,Action,Rockstar Games,Rockstar North,NaN,15.86,9.06,0.06,5.33,1.42,2013-09-17,NaN
4,Call of Duty: Black Ops 3,PS4,Shooter,Activision,Treyarch,8.1,15.09,6.18,0.41,6.05,2.44,2015-11-06,2018-01-14
